Dataset contains Reviews of movies and a binary classification of the movies as output. The words of the reviews are indices of a dictionary.

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

/Users/franzi/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /Users/franzi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/franzi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=1000,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

In [3]:
#one-hot encoding of sentences
tokenizer = Tokenizer(num_words=1000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [4]:
num_classes = 2
y_train2 = to_categorical(y_train, num_classes)
y_test2 = to_categorical(y_test, num_classes)

In [5]:
# Neural Network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(x_train, y_train2, epochs=2, batch_size=100, verbose=1, validation_split=0.2)

# Evaluate the model
score = model.evaluate(x_train, y_train2)
print("\n Training Accuracy:", score[1])
score = model.evaluate(x_test, y_test2)
print("\n Testing Accuracy:", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               128128    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 128,386
Trainable params: 128,386
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 3s 154us/step - loss: 0.4403 - acc: 0.7927 - val_loss: 0.3357 - val_acc: 0.8580
Epoch 2/2
25000/25000 [==============================] - 2s 80us/step
('\n Training Accuracy:', 0.88068)
25000/25000 [==============================] - 2s 87us/step
('\n Testing Accuracy:', 0.85812)


In [6]:
# define model
model = Sequential()
e = Embedding(x_train.shape[1], 100, input_length=x_train.shape[1])
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(x_train, y_train, epochs=50)

# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         100000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 100000)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 100001    
Total params: 200,001
Trainable params: 200,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
25000/25000 [==============================] - 18s 725us/step - loss: 0.3885 - acc: 0.82601s - loss:
Epoch 2/50
25000/25000 [==============================] - 17s 661us/step - loss: 0.3268 - acc: 0.8634
Epoch 3/50
25000/25000 [==============================] - 17s 693us/step - loss: 0.3196 - acc: 0.8668
Epoch 4/50
25000/25000 [==============================] - 16s 655us/step - loss